In [ ]:
import numpy as np
import random

def mod_inverse(det, mod=26):
    """Find modular inverse of determinant under mod 26"""
    det = det % mod
    for i in range(1, mod):
        if (det * i) % mod == 1:
            return i
    return None  # No modular inverse exists

def is_valid_key(matrix):
    """Check if the given 2x2 matrix is invertible under mod 26"""
    det = int(round(np.linalg.det(matrix))) % 26  # Ensure determinant is integer
    return mod_inverse(det, 26) is not None

def generate_invertible_key():
    """Generate a random invertible 2x2 matrix under mod 26"""
    while True:
        a, b, c, d = [random.randint(0, 25) for _ in range(4)]
        key_matrix = np.array([[a, b], [c, d]])
        det = int(round(np.linalg.det(key_matrix))) % 26

        if mod_inverse(det, 26) is not None:
            return key_matrix  # Return only if it's invertible

def encrypt(plain_text, key_matrix):
    """Encrypts plaintext using Hill Cipher"""
    plain_text = plain_text.upper().replace(" ", "")
    padding = False

    if len(plain_text) % 2 != 0:
        plain_text += 'X'  # Padding if odd length
        padding = True  # Mark padding added

    num_text = [ord(char) - ord('A') for char in plain_text]
    cipher_text = ""

    for i in range(0, len(num_text), 2):
        vector = np.array([[num_text[i]], [num_text[i+1]]])
        result = np.dot(key_matrix, vector) % 26
        cipher_text += chr(result[0][0] + ord('A')) + chr(result[1][0] + ord('A'))

    return cipher_text, padding

def decrypt(cipher_text, key_matrix, padding):
    """Decrypts ciphertext using Hill Cipher"""
    cipher_text = cipher_text.upper().replace(" ", "")
    num_text = [ord(char) - ord('A') for char in cipher_text]

    det = int(round(np.linalg.det(key_matrix))) % 26
    det_inv = mod_inverse(det, 26)
    if det_inv is None:
        raise ValueError("Invalid Key: Non-invertible matrix")

    adjugate = np.array([[key_matrix[1][1], -key_matrix[0][1]],
                          [-key_matrix[1][0], key_matrix[0][0]]]) % 26
    key_inv = (det_inv * adjugate) % 26
    key_inv = np.where(key_inv < 0, key_inv + 26, key_inv)  # Ensure positive values

    plain_text = ""

    for i in range(0, len(num_text), 2):
        vector = np.array([[num_text[i]], [num_text[i+1]]])
        result = np.dot(key_inv, vector) % 26
        plain_text += chr(result[0][0] + ord('A')) + chr(result[1][0] + ord('A'))

    if padding:
        plain_text = plain_text.rstrip('X')  # Remove padding only if it was added

    return plain_text

def main():
    print("=== Hill Cipher (2x2) ===")

    while True:
        choice = input("\nChoose an option: (E)ncrypt, (D)ecrypt, (Q)uit: ").upper()

        if choice == 'Q':
            print("Exiting...")
            break
        elif choice not in ['E', 'D']:
            print("Invalid choice! Try again.")
            continue

        # Key matrix selection
        use_random_key = input("Generate a random key? (Y/N): ").upper()
        if use_random_key == 'Y':
            key_matrix = generate_invertible_key()
            print("Generated Key Matrix:\n", key_matrix)
        else:
            print("Enter a 2x2 key matrix (space-separated rows):")
            try:
                key_matrix = np.array([list(map(int, input().split())),
                                       list(map(int, input().split()))])
            except ValueError:
                print("Invalid input! Enter numbers only.")
                continue

            if key_matrix.shape != (2, 2):
                print("Invalid matrix! Enter a 2x2 matrix.")
                continue

            if not is_valid_key(key_matrix):
                print("Invalid key matrix! Not invertible in mod 26.")
                continue

        # Taking text input
        text = input("Enter text: ").upper()

        if choice == 'E':
            encrypted_text, padding = encrypt(text, key_matrix)
            print(f"Encrypted Text: {encrypted_text}")
        elif choice == 'D':
            padding = False  # Assume no padding unless known
            decrypted_text = decrypt(text, key_matrix, padding)
            print(f"Decrypted Text: {decrypted_text}")

if __name__ == "__main__":
    main()


=== Hill Cipher (2x2) ===
Enter a 2x2 key matrix (space-separated rows):
Invalid key matrix! Not invertible in mod 26.
